In [25]:
# imports
from collections import OrderedDict
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from astropy.time import Time
import os
import numpy as np
import math as m
from astropy import units as u
from astropy.table import Table
from astropy.time import Time
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.ticker import ScalarFormatter
from cycler import cycler
import matplotlib.gridspec as gridspec # GRIDSPEC 
from scipy.optimize import curve_fit
from math import cos, pi,sqrt
from astropy.constants import G, c, M_sun
from astropy import units as u
from scipy.stats import spearmanr
from IPython.core.display import display, HTML
import scipy.odr.odrpack as odr
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib qt

In [2]:
def read_comp_diskbb(comp_file):
    data = np.genfromtxt(comp_file, delimiter='\t', names=True, dtype=("U23", "U13", "U14", "U11", 
                                                                       "<f8", "<f8", "<f8", 
                                                                       "<f8", "<f8", "<f8",
                                                                      "<f8", "<f8", "<f8"),
                        converters={5:linked_to_errors, 6:linked_to_errors, 8:linked_to_errors, 9:linked_to_errors, 10: logtoflux, 11: logtoflux_bounds, 12: logtoflux_bounds})

    if len(np.atleast_1d(data))==1:
        data = np.array([data])
    data.sort(order='epoch')
    return data

def read_component_file(file):
    if "disk" in file:
        return read_comp_diskbb(file)
    elif "simpl" in file:
        return read_comp_simpl(file)
    elif "compTT" in file:
        return read_comp_compTT
    else:
        return "error: unknown component file %s" %file

def read_pulsations_file(file):
    data = np.genfromtxt(file, delimiter='\t', names=True, dtype=("U23", "U13", "U14", "U11", 
                                                                       "i8", "f8", "f8"), 
                         missing_values={4:"", 5:"", 6:""}, filling_values={4:-1, 5:-1}, 
                         converters={})
    if len(np.atleast_1d(data))==1:
        data = np.array([data])
    data.sort(order='epoch')
    return data



def pulsations_to_color(pulsations, pulse_fractions):
    color_list = []
    for pulsation, pulse_fraction in zip(pulsations, pulse_fractions):
        if int(pulsation)==-1:
            color = "white"
        elif int(pulsation)==0:
            color = "red"
        elif int(pulsation)==1:
            color = "green"
        else:
            color="white"
        color_list.append(color)
    return np.array(color_list)

def pulse_fraction_to_color(pulse_fraction, pulsations):
    if float(pulsations)==-1:
        return "white"
    elif int(pulsations)==0:
        return "red"
        pulse_fraction * 0.2
    elif int(pulsations)==1:
        return "green"
    cmap = cm.get_cmap('Wistia')
    color = cmap(float(pulse_fraction)/40)
    return color

def pulsation_to_color(pulsation):
    if pulsation==0:
        return "red"
    elif pulsation==1:
        return "green"
    elif pulsation==-1:
        return "white"
    else:
        return "white"
def read_tbabs(comp_file):
    data = np.genfromtxt(comp_file, delimiter='\t', names=True, dtype=("U23", "U11", "U14", "U10", 
                                                                       "<f8", "<f8", "<f8"), 
                         converters={5:linked_to_errors, 6:linked_to_errors})
    if len(np.atleast_1d(data))==1:
        data = np.array([data])
    data.sort(order='epoch')
    return data

def linked_to_errors(x):
    if float(x)==-3:
        print("Linked parameter found")
        return 0.00001
    elif float(x)==-5:
        print("Limit found")
        return 0
    else:
        return x


def read_comp_cutoffpl(comp_file):
    data = np.genfromtxt(comp_file, delimiter='\t', names=True, 
                         dtype=("U23", "U13", "U7", "U14", "<f8", "<f8", "<f8", 
                                "<f8", "<f8", "<f8","<f8", "<f8", "<f8",
                               "<f8", "<f8", "<f8"), 
                         converters={13: logtoflux, 14: logtoflux_bounds, 
                                     15: logtoflux_bounds})
    if len(np.atleast_1d(data))==1:
        data = np.array([data])
    data.sort(order='epoch')
    return data

def read_comp_compTT(comp_file):
    data = np.genfromtxt(comp_file, delimiter='\t', names=True, 
                         dtype=("U23", "U13", "U7", "U14", "<f8", "<f8", "<f8", 
                          "<f8", "<f8", "<f8", "<f8", "<f8", "<f8", "<f8", "<f8", "<f8",
                          "<f8", "<f8", "<f8", "<f8", "<f8", "<f8","<f8", "<f8", "<f8"), 
                         converters={22: logtoflux, 23: logtoflux_bounds, 
                                     24: logtoflux_bounds})
    if len(np.atleast_1d(data))==1:
        data = np.array([data])
    data.sort(order='epoch')
    return data

def read_comp_simpl(comp_file):
    data = np.genfromtxt(comp_file, delimiter='\t', names=True, 
                         dtype=("U23", "U13", "U7", "U14", "<f8", "<f8", "<f8", "<f8", "<f8", 
                                "<f8", "<f8", "<f8", "<f8", "<f8", "<f8", "<f8"), 
                         converters={5:linked_to_errors, 6:linked_to_errors, 8:linked_to_errors, 
                                     9:linked_to_errors, 13:logtoflux, 14:logtoflux_bounds, 15:logtoflux_bounds})
    if len(np.atleast_1d(data))==1:
        data = np.array([data])
    data.sort(order='epoch')
    return data

def tojd (x):
    return Time(x).jd
     
def logtoflux_bounds(x):
    # keep upper and lower bounds to 0
    if float(x)==0:
        print("Flux bound found")
        return 0
    elif float(x)==-1:
        print("Warning; error in flux computation detected")
        return 0
    else:
        return m.pow(10,float(x))
    
def logtoflux(x):
    if float(x)==-1:
        print("Warning; error in flux computation detected")
        return 0
    return m.pow(10,float(x))

def read_flux_data_old(flux_file):
    data = np.genfromtxt(flux_file, delimiter='\t', names=True, 
                         dtype=("U23", "U13", "U7", "U12", "<f8", "<f8", "<f8", "<f8", "<f8", "<f8", 
                                "<f8", "<f8", "<f8", "<f8", "<f8", "<f8"), 
                         missing_values='Error',
                         converters={4: logtoflux, 5: logtoflux_bounds, 6: logtoflux_bounds, 
                                    7: logtoflux, 8: logtoflux_bounds, 
                                   9: logtoflux_bounds, 10: logtoflux, 11: logtoflux_bounds, 12: logtoflux_bounds,
                                    13: logtoflux, 14: logtoflux_bounds, 15: logtoflux_bounds}, 
                         filling_values=0)
    data.sort(order='epoch')
    return data

def read_flux_data(flux_file):
    data = np.genfromtxt(flux_file, delimiter='\t', names=True, 
                         dtype=("U23", "U13", "U7", "U12", "<f8", "<f8", "<f8", 
                                "<f8", "<f8", "<f8", "<f8", "<f8", "<f8", "<f8", "<f8", "<f8", 
                                "<f8", "<f8", "<f8", "<f8", "<f8", "<f8", "<f8", "<f8", "<f8"), 
                         missing_values='Error',
                         converters={4: logtoflux, 5: logtoflux_bounds, 6: logtoflux_bounds, 
                                    7: logtoflux, 8: logtoflux_bounds, 
                                   9: logtoflux_bounds, 10: logtoflux, 11: logtoflux_bounds, 12: logtoflux_bounds,
                                    13: logtoflux, 14: logtoflux_bounds, 15: logtoflux_bounds,
                                    16: logtoflux, 17: logtoflux_bounds, 18: logtoflux_bounds, 
                                    19: logtoflux, 20: logtoflux_bounds, 21: logtoflux_bounds, 
                                    22: logtoflux, 23: logtoflux_bounds, 24: logtoflux_bounds}, 
                         filling_values=0)
    data.sort(order='epoch')
    return data

def remove_legend_repetitions(ax, fontsize=20):
    """Removes any repetead entries in the legend. Adds the legend to the plot too.
    Parameters
    ----------
    ax : The axis were the repeated entries are to be removed. """
    handles, labels = ax.get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), loc='best', prop={'size': fontsize})

def bounds_to_errors(values, lowerbounds, upperbounds):
    ''' Compute errors given the lower and upper bounds of a an array of values.
    Parameters:
    -----------
    value : the central values given by the fit
    lowerbound : its lower bounds
    upperbound : its upper bounds'''

    lower_errors = values - lowerbounds
    upper_errors = upperbounds - values
    
    for value, lowerbound, upperbound in zip(values, lowerbounds, upperbounds):
        
        if upperbound<value and upperbound!=0:
            print("Warning upperbound is lower than value!!! %.5f < %.5f" % (upperbound, value))
        if lowerbound>value and lowerbound!=0:
            print("Warning lowerbound is higher than value!!! %.5f > %.5f" % (lowerbound, value))
            
    
    uplims = np.zeros(lower_errors.shape)
    # lower bound (upper limit)
    uplims[np.where(lowerbounds==0)] = 1
    lower_errors[np.where(lowerbounds==0)] = (upper_errors[np.where(lowerbounds==0)] - values[np.where(lowerbounds==0)]) * 0.2
    values[np.where(lowerbounds==0)] = upperbounds[np.where(lowerbounds==0)]
    upper_errors[np.where(lowerbounds==0)] = 0
    
    # upper bound found (lower limit)
    lolims = np.zeros(lower_errors.shape)
    lolims[np.where(upperbounds==0)] = 1
    upper_errors[np.where(upperbounds==0)] = (values[np.where(upperbounds==0)] - lower_errors[np.where(upperbounds==0)]) * 0.2
    values[np.where(upperbounds==0)] = lowerbounds[np.where(upperbounds==0)]
    lower_errors[np.where(upperbounds==0)] = 0
    return lower_errors, upper_errors, lolims, uplims

def jd_to_daymonthyear(x, pos):
    '''Format the axis to convert from Julian day to real date.'''
    time = Time(x, format='jd')
    time.format = 'iso'
    time.out_subfmt = 'date'
    return time

def compute_ratios(hard_flux, hard_flux_low, hard_flux_high, soft_flux, soft_flux_low, soft_flux_high):
    """Compute hardness ratios given fluxes in two energy bands."""
    ratio = hard_flux / soft_flux
    soft_err_low = soft_flux - soft_flux_low
    soft_err_high = soft_flux_high - soft_flux
    hard_err_low = hard_flux - hard_flux_low
    hard_err_high = hard_flux_high - hard_flux
    ratio_err_low = ((hard_err_low / soft_flux)**2 + (hard_flux * soft_err_low / soft_flux**2)**2)**(1 / 2)
    ratio_err_high = ((hard_err_high / soft_flux)**2 + (hard_flux * soft_err_high / soft_flux**2)**2)**(1 / 2)
    return ratio, ratio_err_low, ratio_err_high


def create_color_array(data_length, cmap='hsv'):
    """Create an array of colors given the length of a dataset. Useful for plots where a unique color is needed for each dataset.

    The returned colors come from the input map (jet by default).

    Parameters
    ----------
    data_length : The length of your data for the color array creation.

    """
    print("Creating color array for %i datasets" % data_length)
    x = np.arange(data_length)
    ys = [i + x + (i * x)**2 for i in range(data_length)]
    setmap = plt.get_cmap(name=cmap)

    colors = setmap(np.linspace(0, 1, len(ys)))
    return colors


def get_markers_array(data_length):
    """Get an array of markers given the length of a dataset. Useful for plots where a unique marker is needed for each dataset.

    There are 17 different markers and after that they are repeated.

    Parameters
    ----------
    data_length : The length of your data for the marker array creation.

    """
    m = ['o', '^', (12, 1, 50), "s", 'v', 'p', 'P',  'd', '*', 'h', (5, 1, 10), (5, 0, 10), 'D', '8' , (10, 1, 20), 
         '<', 'x', '.', '>', (7, 0, 30), (20, 0, 50), '$u$', '1']

    while data_length > len(m):
        m.extend(m)

    return m

def draw_arrows(x, y, colors, ax=None):
    for i in np.arange(1, len(x)):
        if ax==None:
            plt.annotate("", xy=(x[i-1],y[i-1]), xytext=(x[i], y[i]), arrowprops=dict(arrowstyle="<-", shrinkA=10, shrinkB=10, color=colors[i-1]))
        else:
            ax.annotate("", xy=(x[i-1],y[i-1]), xytext=(x[i], y[i]), arrowprops=dict(arrowstyle="<-", shrinkA=10, shrinkB=10, color=colors[i-1]))
def fit_diskbb(fluxes, temperatures, temperatures_errlow, temperatures_errhigh, b=4):
    popt, pconv=curve_fit(diskLvsT, temperatures, fluxes,  
                          sigma=(temperatures_errlow + temperatures_errhigh)/2, 
                          p0=[10**-13, b])
    a, b = popt
    a_err, b_err = np.sqrt(np.diag(pconv))
    return a, b, a_err, b_err

def fit_diskbb_odr(x, x_errlow, x_errhigh, y, y_errlow, y_errhigh, index=4, norm=5):
    model = odr.Model(diskLvsT_odr)
    data = odr.Data(x, y, wd=1./((x_errlow+x_errhigh)/2)**2, we=1./((y_errlow+y_errhigh)/2)**2)
    myodr = odr.ODR(data, model, beta0=[norm, index])
    output = myodr.run()
    return output.beta[0], output.beta[1], output.sd_beta[0], output.sd_beta[1]

def fit_outflow(luminosity, temperatures, luminosity_errlow, luminosity_errhigh):
    popt, pconv=curve_fit(diskLvsT, temperatures, luminosity,  
                          sigma=(luminosity_errlow + luminosity_errhigh)/2, 
                          p0=[1.74,-1])
    a, b = popt
    a_err, b_err = np.sqrt(np.diag(pconv))
    return a, b, a_err, b_err

def diskLvsT(x, a, b):
            return a * x**b
def diskLvsT_odr(B, x):
            return B[0] * x**B[1]
def radius_tomass(r_in):
    return r_in * (c.to("km/s")) ** 2 / 2 / G.to("km**3/kg/s**2") / M_sun
def disknorm_tomass(norm, distance, angle=60): 
    r_in = disknorm_tosize(norm, distance, angle)
    return radius_tomass(r_in)
def disknorm_tosize(norm, distance, angle=60):
    angle_rad = 60 / 360 * 2 * pi
    # mega parsecs to parsecs
    distance = distance * 10 ** 3 
    r_in = np.sqrt(norm/cos(angle_rad)) * (distance / (10)) * u.km
    return r_in
                         
def plotdisk(data, ax, param_1="Tin", param_2="flux"):
    if len(np.atleast_1d(data))>1:
        colors = create_color_array(len(data["epoch"]), "jet")
        markers = get_markers_array(len(data["epoch"]))
    else:
        colors = ["cyan"]
        markers = ['x']
    ax.set_prop_cycle(cycler('color', colors))
    param_1err_low, param_1err_high, param_1lolimits, param_1uplimits = bounds_to_errors(data["%s" %param_1], 
                                                                                         data["%slow" %param_1], 
                                                                                         data["%supp" %param_1])
    param_2err_low, param_2err_high, param_2lolimits, param_2uplimits = bounds_to_errors(data["%s" %param_2], 
                                                                                         data["%slow" %param_2], 
                                                                                         data["%supp" %param_2])
    for index in np.arange(0, len(data["epoch"])):
        if data["xmm_obsid"][index]!="":
            label=data["xmm_obsid"][index]
        else:
            label=data["chandra"][index]

        ax.errorbar(data["%s" %param_1][index], data["%s" %param_2][index] , 
                             xerr=[[param_1err_low[index]], [param_1err_high[index]]], 
                             yerr=[[param_2err_low[index]], [param_2err_high[index]]], label=label, 
                             marker="$ f $", xlolims=param_1lolimits[index], xuplims=param_1uplimits[index], 
                             uplims=param_2uplimits[index], lolims=param_2lolimits[index], markersize=10)
def eddington_limit(M):
    return 1.26 * M * 10**38 / 10**39
def bolometric_l(M, m_dot):
    return eddington_limit(M) * (1 + 3/5 * np.log(m_dot))
def t_disk_max(M, m_dot):
    return 1.6 * (M)**(-1/4) * (1 - 0.2 * m_dot**(-1/3))
def t_spherization_max(M, m_dot):
    return 1.5 * (M)**(-1/4) * m_dot**(-1/2) * (1 + 0.3 * m_dot**(-3/4))
def t_photosphere_max(M, m_dot, beta=1, chi=1, epsilon_wind=1/2):
    return 0.8 * (beta * chi / epsilon_wind)**1/2 * M**(-1/4) * m_dot**(-3/4)

In [3]:
def readbroadbandfile(file="broadband_fitting_plot.config"):
    broadband_file = "/home/agurpide/x_ray_data/%s" %file
    plot_config = np.genfromtxt(broadband_file, delimiter="\t\t", dtype=("U13", "U30", "U18", float, float), names=True)
    print(plot_config.dtype)
    return plot_config

In [31]:
def create_plot(xlabel, ylabel, title=""):
    # one column plot figure size
    figure, ax = plt.subplots(1, 1, figsize=(16, 12), edgecolor="black")
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    return figure, ax
print(plt.style.available)
plt.style.use('/home/agurpide/.config/matplotlib/stylelib/presentation.mplstyle')

['seaborn-whitegrid', 'seaborn-poster', 'seaborn-colorblind', 'seaborn-deep', 'tableau-colorblind10', 'seaborn-dark', 'seaborn-talk', 'seaborn-muted', 'seaborn-dark-palette', 'seaborn-pastel', 'bmh', 'seaborn', 'grayscale', 'seaborn-notebook', 'fast', 'seaborn-ticks', 'seaborn-darkgrid', 'Solarize_Light2', 'fivethirtyeight', 'classic', 'ggplot', 'seaborn-white', 'seaborn-bright', 'dark_background', '_classic_test', 'seaborn-paper']


## Rin vs L tot

In [125]:
plt.rc('font', family='serif')
plt.rc('xtick', labelsize='x-small')
plt.rc('ytick', labelsize='x-small')
plot_config = readbroadbandfile(file="broadband_fitting_plot.config")
include_chandra_data = 0
common_dir = "/home/agurpide/x_ray_data/"
param_1 = "norm"
comp_1 = "diskbb_1"
model_components = ["diskbb_0", "diskbb_1"]
total_flux = "0325"
for source, source_label, model_dir, distance in zip(plot_config["source_dir"], 
                                                     plot_config["source_name"], 
                                                     plot_config["model_dir"], 
                                                     plot_config["distance"]):
    
    figure_disk, disk_size_l_ax = create_plot("Soft diskbb size (km)", 
                                              "L [0.3 - 25] (10$^{39}$) erg/s)", title=source_label)
    
    print("Plotting source %s" % source)
    parsecs = distance * m.pow(10, 6)
    # parsecs to cm
    distancecm = u.pc.to(u.cm, parsecs)
    constant = 4 * m.pi * distancecm ** 2 / 10**39
    source_dir = "%s/%s/%s" %(common_dir, source, model_dir)
    flux_data = read_flux_data("%s/fluxes/fluxes.dat" % source_dir)

    if not os.path.isfile("%s/components/%s.dat" % (source_dir, comp_1)):
        print("Component %s not found in this source" % component)
        continue
    component_data = read_comp_diskbb("%s/components/%s.dat" % (source_dir, comp_1))
    if os.path.isfile("%s/components/chandra_%s.dat" % (source_dir, comp_1)):
        chandra_disk_data = read_comp_diskbb("%s/components/chandra_%s.dat" % (source_dir, comp_1))
        if include_chandra_data:
            component_data = np.append(component_data, chandra_disk_data)
    flux_data_current = np.array([flux for flux in flux_data if flux["xmm_obsid"] in 
                                  component_data["xmm_obsid"] and flux["chandra"] in 
                                  component_data["chandra"]])
    markers = get_markers_array(len(flux_data_current["epoch"]))
    print("Found %d observations" %len(flux_data_current))
    colors = create_color_array(len(flux_data_current["epoch"]), "jet")
    param_1err_low, param_1err_high, param_1lolimits, param_1uplimits = bounds_to_errors(disknorm_tosize(component_data["%s" %param_1], distance), 
                                                                                         disknorm_tosize(component_data["%slow" %param_1], distance), 
                                                                                         disknorm_tosize(component_data["%supp" %param_1], distance))

    for index, color, marker in zip(np.arange(0, len(flux_data_current["epoch"])), colors, markers):
        if flux_data_current["xmm_obsid"][index]!="":
            label=flux_data_current["epoch"][index]
            facemarkercolor = color
            print("Plotting xmm epoch %s" %label)
        else:
            label=flux_data_current["epoch"][index]
            print("Plotting chandra epoch %s" %label)
            facemarkercolor = "None"
        disk_size = disknorm_tosize(component_data["%s" %param_1], distance).value
        disk_size_l_ax.errorbar(disk_size[index], 
                                flux_data_current["%s"%total_flux][index] * constant , 
                             xerr=[[param_1err_low[index].value], [param_1err_high[index].value]], 
                             yerr=[[constant * (flux_data_current["%s"%total_flux] - flux_data_current["%s_lower" %total_flux])[index]],
                   [constant * (flux_data_current["%s_upper" %total_flux]-flux_data_current["%s"%total_flux])[index]]], 
                             label=label, marker=marker, xlolims=param_1lolimits[index], 
                             xuplims=param_1uplimits[index], 
                            markersize=10, color=color, markerfacecolor=facemarkercolor)
    plt.legend()
    outdir = "/home/agurpide/Documents/paper1/%s" % source_label
    if not os.path.isdir(outdir):
        os.mkdir(outdir)
    figure_disk.savefig("%s/%sdisk_vstotalL.png" %(outdir, comp_1), bbox_inches='tight')   

[('source_dir', '<U13'), ('model_dir', '<U30'), ('source_name', '<U18'), ('distance', '<f8')]
Plotting source holmbergIIX1
Found 7 observations
Creating color array for 7 datasets
Plotting xmm epoch 2002-04-10
Plotting xmm epoch 2002-04-16
Plotting xmm epoch 2002-09-18
Plotting xmm epoch 2004-04-15
Plotting xmm epoch 2010-03-26
Plotting xmm epoch 2013-09-09
Plotting xmm epoch 2013-09-17
Plotting source holmbergX9
Limit found
Found 10 observations
Creating color array for 10 datasets
Plotting xmm epoch 2002-04-10
Plotting xmm epoch 2002-04-16
Plotting xmm epoch 2004-09-26
Plotting xmm epoch 2011-03-24
Plotting xmm epoch 2011-09-26
Plotting xmm epoch 2011-11-23
Plotting xmm epoch 2012-10-23
Plotting xmm epoch 2012-10-25
Plotting xmm epoch 2012-11-12
Plotting xmm epoch 2012-11-14
Plotting source NGC5204X-1
Found 8 observations
Creating color array for 8 datasets
Plotting xmm epoch 2003-01-06
Plotting xmm epoch 2003-04-25
Plotting xmm epoch 2003-05-01
Plotting xmm epoch 2006-11-16
Plotting

# Disk flux vs T

In [32]:
plot_config = readbroadbandfile(file="broadband_fitting_plot.config")
if len(np.atleast_1d(plot_config))==1:
        plot_config = np.array([plot_config])
common_dir = "/home/agurpide/x_ray_data/"
param_1 = "Tin"
param_2 = 'flux'
plot_out_name = "disk_flux_t"
comp= "diskbb_0"
include_chandra_data = 1
correlations_file_path = "/home/agurpide/x_ray_data/%s_L.dat" %comp
if not os.path.isfile(correlations_file_path):
    correlations_file= open("%s" %correlations_file_path,"a+")
    correlations_file.write("#source\tspearman\tpvalue\tindex\terror\n")
else:
    correlations_file= open("%s" %correlations_file_path,"a")
for source, source_label, model_dir, distance in zip(plot_config["source_dir"], 
                                                     plot_config["source_name"], 
                                                     plot_config["model_dir"], 
                                                     plot_config["distance"]):

    print("Plotting source %s component %s" % (source, comp))
    flux_diskbb, diskbb_T_ax = create_plot("%s keV" %param_1, 
                                           "L$_{bol}$ soft diskbb (10$^{39}$ erg/s)",
                                           source_label)

    parsecs = distance * m.pow(10, 6)
    # parsecs to cm
    distancecm = u.pc.to(u.cm, parsecs)
    constant = 4 * m.pi * distancecm ** 2 / 10**39
    source_dir = "%s/%s/%s" %(common_dir, source, model_dir)
    if source=="M82X-1":
        data = read_comp_diskbb("%s/components/chandra_diskbb_0.dat" %source_dir)
    else:
        data = read_comp_diskbb("%s/components/%s.dat" % (source_dir, comp))
        print("Found XMM-Newton %d observations" % len(data))
        if os.path.isfile("%s/components/chandra_%s.dat" %(source_dir, comp)):
            chandra_data = read_comp_diskbb("%s/components/chandra_%s.dat" % (source_dir, comp))
            print("Found Chandra %d observations" % len(np.atleast_1d(chandra_data)))
            if include_chandra_data:
                data = np.append(data, chandra_data)

    data.sort(order="epoch")
    colors = create_color_array(len(data["epoch"]))
    markers = get_markers_array(len(data["epoch"]))
    param_1err_low, param_1err_high, param_1lolimits, param_1uplimits = bounds_to_errors(data["%s" %param_1], 
                                                                                         data["%slow" %param_1], 
                                                                                         data["%supp" %param_1])
    if param_2=='flux':
        data["%s" %param_2] = data["%s" %param_2] *constant
        data["%slow" %param_2] = data["%slow" %param_2] *constant
        data["%supp" %param_2] = data["%supp" %param_2] *constant
        
    param_2err_low, param_2err_high, param_2lolimits, param_2uplimits = bounds_to_errors(data["%s" %param_2], 
                                                                                         data["%slow" %param_2], 
                                                                                         data["%supp" %param_2])
    for index, color in enumerate(colors):
        if data["xmm_obsid"][index]!="":
                label=data["epoch"][index]
                if data["nustar_obsid"][index]!="":
                    facemarkercolor="white"
                else:
                    facemarkercolor=color
        else:
            label=data["epoch"][index]
            facemarkercolor = "black"
            
        print("Plotting observation %s" %label)
        diskbb_T_ax.errorbar(data["%s" %param_1][index], data["%s" %param_2][index], 
                             xerr=[[param_1err_low[index]], [param_1err_high[index]]], 
                             yerr=[[param_2err_low[index]], [param_2err_high[index]]], 
                             label=label, color=color,
                             marker=markers[index], xlolims=param_1lolimits[index], 
                             xuplims=param_1uplimits[index], 
                             uplims=param_2uplimits[index], 
                             markerfacecolor=facemarkercolor, 
                             lolims=param_2lolimits[index], markersize=10)

    #draw_arrows(data["%s" %param_1], data["%s" %param_2], colors)
    calculate_correlation = 1
    rho, p = spearmanr(data["%s" %param_2], data["%s" %param_1])
    out_string = "%s\t%.1f\t%.5f" % (source_label, rho,p)
    #diskbb_T_ax.set_title("%s (%.1f)" %(source_label, rho))    
    if calculate_correlation==1:

        a, b, a_err, b_err = fit_diskbb_odr(data["%s" %param_1], param_1err_low, param_1err_high, data["%s" %param_2], 
                                        param_2err_low, param_2err_high, index=2 * rho, norm=5)
        out_string = "%s\t%.1f\t%.1f\n" %(out_string, b, b_err)
        #print("%.1f $\pm$ %.2f" % (b, b_err))

        data.sort(order='Tin')
        plt.plot(data["%s" %param_1], diskLvsT(data["%s" %param_1], a ,b), 
                 color="black", marker=None, ls='solid', label='L $\propto$ T$^{%.1f \pm %.1f}$' %(b,b_err))
        min_index = 2
        #plt.plot(data["%s" %param_1], diskLvsT(data["%s" %param_1], data["%s" %param_2][min_index]/(data["%s" %param_1][min_index]**4) ,4), 
        #         color="blue", marker=None, ls='--', label='L $\propto$ T$^4$')
    else:
        out_string = "%s\t\t\n" %out_string
    correlations_file.write(out_string)
    
    plot_diskbb = 0
    if plot_diskbb==1:
        data.sort(order="Tin")
        plt.plot(data["%s" %param_1], diskLvsT(
            data["%s" %param_1], data["%s" %param_2][0]/(data["%s" %param_1][0]**4) ,4),
                 color="blue", marker=None, ls='--', label='L $\propto$ T$^4$')
        temperatures = np.arange(np.min(data["%s" %param_1]), np.max(data["%s" %param_1]), 0.001)
        plt.plot(temperatures, diskLvsT(temperatures, data["%s" %param_2][-1]/(data["%s" %param_1][-1]**4) ,4),
                 color="blue", marker=None, ls='--', label='L $\propto$ T$^4$')
    

    diskbb_T_ax.legend()
    formatter_major = FuncFormatter(lambda y, _: '%.0f'%y)
    formatter_minor = FuncFormatter(lambda y, _: '%.1f'%y)
    
    diskbb_T_ax.set_yscale("log")
    diskbb_T_ax.set_xscale("log")
    diskbb_T_ax.yaxis.set_major_formatter(formatter_major)
    diskbb_T_ax.yaxis.set_minor_formatter(formatter_minor)
    diskbb_T_ax.xaxis.set_minor_formatter(mticker.ScalarFormatter())
    diskbb_T_ax.xaxis.set_major_formatter(mticker.ScalarFormatter())
    outdir = "/home/agurpide/Documents/paper1/%s" % source_label
    if not os.path.isdir(outdir):
        os.mkdir(outdir)
    flux_diskbb.savefig("%s/%s%s%s.png" % (outdir, source, comp, plot_out_name), bbox_inches='tight')   
correlations_file.close()

[('source_dir', '<U13'), ('model_dir', '<U30'), ('source_name', '<U18'), ('distance', '<f8'), ('hardness', '<f8')]
Plotting source CircinusULX5 component diskbb_0
Found XMM-Newton 6 observations
Found Chandra 1 observations
Creating color array for 7 datasets
Plotting observation 2001-08-06
Plotting observation 2010-12-24
Plotting observation 2013-02-03
Plotting observation 2014-03-01
Plotting observation 2016-08-23
Plotting observation 2018-02-07
Plotting observation 2018-09-16
Plotting source NGC6946X-1 component diskbb_0
Found XMM-Newton 4 observations
Found Chandra 5 observations
Creating color array for 9 datasets
Plotting observation 2001-09-07
Plotting observation 2004-10-22
Plotting observation 2004-11-06
Plotting observation 2004-12-03
Plotting observation 2007-11-02
Plotting observation 2007-11-08
Plotting observation 2012-10-21
Plotting observation 2016-09-28
Plotting observation 2017-06-01
Plotting source NGC6946X-1 component diskbb_0
Found XMM-Newton 5 observations
Found C

## L soft component vs L hard component

In [114]:
plot_config = readbroadbandfile(file="broadband_fitting_plot.config")
if len(np.atleast_1d(plot_config))==1:
        plot_config = np.array([plot_config])
common_dir = "/home/agurpide/x_ray_data/"
param_1 = "flux"
param_2 = "flux"
plot_out_name = "flux_soft_hard"
model_components = ["diskbb_0", "diskbb_1"]

for source, source_label, model_dir, distance in zip(plot_config["source_dir"], plot_config["source_name"], plot_config["model_dir"], plot_config["distance"]):
    
        print("Plotting source %s" % source)
        fluxes_components_figure, fluxes_ax = create_plot("Flux soft component [0.01 - 100 keV] (erg/s/cm$^2$)", 
                                           "Flux hard component flux [0.01 - 100 keV] (erg/s/cm$^2$)",
                                           source_label)
        parsecs = distance * m.pow(10, 6)
        # parsecs to cm
        distancecm = u.pc.to(u.cm, parsecs)
        constant = 4 * m.pi * distancecm ** 2 / 10**39
        source_dir = "%s/%s/%s" %(common_dir, source, model_dir)
        
        if not os.path.isfile("%s/components/%s.dat" % (source_dir, model_components[0])) or not os.path.isfile("%s/components/%s.dat" % (source_dir, model_components[1])):
            print("Both %s and %s components not found in this source" % (model_components[0],
                                                                         model_components[1]))
            continue

        soft_comp_data = read_component_file("%s/components/%s.dat" % (source_dir, model_components[0]))
        hard_comp_data = read_component_file("%s/components/%s.dat" % (source_dir, model_components[1]))
        
         
        if (os.path.isfile("%s/components/chandra_%s.dat" % (source_dir, model_components[0])) and os.path.isfile("%s/components/chandra_%s.dat" % (source_dir, model_components[1]))):
            chandra_soft_data = read_component_file("%s/components/chandra_%s.dat" % (source_dir, model_components[0]))
            chandra_hard_data = read_component_file("%s/components/chandra_%s.dat" % (source_dir, model_components[1]))
            if include_chandra_data==1:
                soft_comp_data = np.append(soft_comp_data, chandra_soft_data)
                hard_comp_data = np.append(hard_comp_data, chandra_hard_data)
            
        # filter obs that have both disk and outflow
        soft_comp_data = np.array([outflow for outflow in soft_comp_data 
                                 if outflow["xmm_obsid"] in hard_comp_data["xmm_obsid"] 
                                   and outflow["chandra"] in hard_comp_data["chandra"]])
        hardcomp_err_low, hardcomp_err_high, hardcomp_lolimits, hardcomp_uplimits = bounds_to_errors(hard_comp_data["%s" %param_1], 
                                                                                                 hard_comp_data["%slow" %param_1], 
                                                                                                 hard_comp_data["%supp" %param_1])

            
        softcomp_err_low, softcomp_err_high, softcomp_lolimits, softcomp_uplimits = bounds_to_errors(soft_comp_data["%s" %param_1], 
                                                                                                 soft_comp_data["%slow" %param_1], 
                                                                                                 soft_comp_data["%supp" %param_1])

        colors = create_color_array(len(hard_comp_data["epoch"]), "jet")
        markers = get_markers_array(len(hard_comp_data["epoch"]))
        fluxes_ax.set_prop_cycle(cycler('color', colors))
        for index, color, marker in zip(np.arange(0, len(hard_comp_data["epoch"])), colors, markers):
            if hard_comp_data["xmm_obsid"][index]!="":
                label=hard_comp_data["xmm_obsid"][index]
            else:
                label=hard_comp_data["chandra"][index]
            
            fluxes_ax.errorbar(soft_comp_data["%s" %param_1][index], hard_comp_data["%s" %param_1][index], 
                                     xerr=[[softcomp_err_low[index]], [softcomp_err_high[index]]], 
                                     yerr=[[hardcomp_err_low[index]],
                           [hardcomp_err_high[index]]], label=label, lolims=hardcomp_lolimits[index], 
                                 uplims=hardcomp_uplimits[index],
                                     marker=marker, xlolims=softcomp_lolimits[index], 
                               xuplims=softcomp_uplimits[index], 
                                    markersize=10, color=color)
        
        fluxes_ax.legend()
        outdir = "/home/agurpide/Documents/paper1/%s" % source_label
        if not os.path.isdir(outdir):
            os.mkdir(outdir)
        fluxes_components_figure.savefig("%s/%s.png" % (outdir, plot_out_name), bbox_inches='tight') 

[('source_dir', '<U13'), ('model_dir', '<U30'), ('source_name', '<U18'), ('distance', '<f8'), ('hardness', '<f8')]
Plotting source M51-ULX8
Creating color array for 11 datasets


## L vs time

In [131]:
plot_config = readbroadbandfile(file="broadband_fitting_plot.config")
if len(np.atleast_1d(plot_config))==1:
        plot_config = np.array([plot_config])
common_dir = "/home/agurpide/x_ray_data/"
plot_out_name = "L_time"
color_bands=["red", "orange", "blue", "green"]
bands = ["0320", "2010", "1025", "0325"]
for source, source_label, model_dir, distance in zip(plot_config["source_dir"], plot_config["source_name"], plot_config["model_dir"], plot_config["distance"]):
    
        print("Plotting source %s" % source)
        flux_time_figure, flux_time_ax = create_plot("Date", 
                                           "L (erg/s)",
                                           source_label)
        parsecs = distance * m.pow(10, 6)
        # parsecs to cm
        distancecm = u.pc.to(u.cm, parsecs)
        constant = 4 * m.pi * distancecm ** 2 / 10**39
        source_dir = "%s/%s/%s" %(common_dir, source, model_dir)
        flux_data = read_flux_data("%s/fluxes/fluxes.dat" % source_dir)
        markers = get_markers_array(len(flux_data["epoch"]))
        print("Found %d observations" %len(flux_data))
        fluxes_ax.set_prop_cycle(cycler('color', colors))
        for band, color in zip(bands, color_bands):
            label="Flux %s" % band
            flux_time_ax.errorbar(Time(flux_data["epoch"]).jd, flux_data["%s"%band] * constant,
                                  label=label, marker=".", markersize=10, 
                                  yerr=[constant * (flux_data["%s" %band]-flux_data["%s_lower"%band]), 
                       constant * (flux_data["%s_upper" %band]-flux_data["%s" %band])],color=color, ls='solid')
        remove_legend_repetitions(flux_time_ax, 14)
        date_formatter = FuncFormatter(jd_to_daymonthyear)
        plt.xticks(rotation=45)
        # set formatter
        flux_time_ax.xaxis.set_major_formatter(date_formatter)
        outdir = "/home/agurpide/Documents/paper1/%s" % source_label
        if not os.path.isdir(outdir):
            os.mkdir(outdir)
        flux_time_figure.savefig("%s/%s.png" % (outdir, plot_out_name), bbox_inches='tight') 

[('source_dir', '<U13'), ('model_dir', '<U30'), ('source_name', '<U18'), ('distance', '<f8'), ('hardness', '<f8')]
Plotting source CircinusULX5
Found 7 observations
Plotting source M81-X6
Found 9 observations


## Hardness ratio vs time

In [19]:
plot_config = readbroadbandfile(file="broadband_fitting_plot.config")
if len(np.atleast_1d(plot_config))==1:
        plot_config = np.array([plot_config])
common_dir = "/home/agurpide/x_ray_data/"
plot_out_name = "ratio_time"
soft_band = "0320"
hard_band = "2010"

for source, source_label, model_dir, distance in zip(plot_config["source_dir"], plot_config["source_name"], plot_config["model_dir"], plot_config["distance"]):
    
        print("Plotting source %s" % source)
        ratio_time_figure, ratio_time_ax = create_plot("Date", 
                                           "F [2 - 10 keV] / F [0.3 - 2]",
                                           source_label)
        parsecs = distance * m.pow(10, 6)
        # parsecs to cm
        distancecm = u.pc.to(u.cm, parsecs)
        constant = 4 * m.pi * distancecm ** 2 / 10**39
        source_dir = "%s/%s/%s" %(common_dir, source, model_dir)
        flux_data = read_flux_data("%s/fluxes/fluxes.dat" % source_dir)
        ratio_soft, ratio_err_low_soft, ratio_err_high_soft = compute_ratios(flux_data["%s" %hard_band], flux_data["%s_lower"%hard_band] ,
                       flux_data["%s_upper" %hard_band], flux_data["%s" %soft_band], 
                       flux_data["%s_lower" %soft_band],flux_data["%s_upper"%soft_band])

        markers = get_markers_array(len(flux_data["epoch"]))
        colors = create_color_array(len(flux_data["epoch"]))
        print("Found %d observations" %len(flux_data))
        ratio_time_ax.set_prop_cycle(cycler('color', colors))
        ratio_time_ax.errorbar(Time(flux_data["epoch"]).jd, ratio_soft, marker=".", markersize=10, 
                                  yerr=[ratio_err_low_soft, 
                       ratio_err_high_soft],color=color, ls='solid')
        date_formatter = FuncFormatter(jd_to_daymonthyear)
        # set formatter
        ratio_time_ax.xaxis.set_major_formatter(date_formatter)
        outdir = "/home/agurpide/Documents/paper1/%s" % source_label
        if not os.path.isdir(outdir):
            os.mkdir(outdir)
        ratio_time_figure.savefig("%s/%s.png" % (outdir, plot_out_name), bbox_inches='tight') 

[('source_dir', '<U13'), ('model_dir', '<U30'), ('source_name', '<U18'), ('distance', '<f8'), ('hardness', '<f8')]
Plotting source holmbergX9
Creating color array for 10 datasets
Found 10 observations


## L vs hardness ratio

In [18]:
plot_config = readbroadbandfile(file="broadband_fitting_plot.config")
if len(np.atleast_1d(plot_config))==1:
        plot_config = np.array([plot_config])
common_dir = "/home/agurpide/x_ray_data/"
plot_out_name = "ratio_L"

soft_band = "0320"
hard_band = "2010"
total_band = "03100"

for source, source_label, model_dir, distance in zip(plot_config["source_dir"], plot_config["source_name"], plot_config["model_dir"], plot_config["distance"]):
    
        print("Plotting source %s" % source)
        l_ratio_figure, l_ratio_ax = create_plot("F [2 - 10 keV] / F [0.3 - 2]", 
                                           "L [0.3 - 10] keV (10$^{39}$ erg/s)",
                                           source_label)
        parsecs = distance * m.pow(10, 6)
        # parsecs to cm
        distancecm = u.pc.to(u.cm, parsecs)
        constant = 4 * m.pi * distancecm ** 2 / 10**39
        source_dir = "%s/%s/%s" %(common_dir, source, model_dir)
        flux_data = read_flux_data("%s/fluxes/fluxes.dat" % source_dir)
        ratio_soft, ratio_err_low_soft, ratio_err_high_soft = compute_ratios(flux_data["%s" %hard_band], 
                                                                             flux_data["%s_lower"%hard_band] ,
                       flux_data["%s_upper" %hard_band], flux_data["%s" %soft_band], 
                       flux_data["%s_lower" %soft_band],flux_data["%s_upper"%soft_band])

        markers = get_markers_array(len(flux_data["epoch"]))
        colors = create_color_array(len(flux_data["epoch"]))
        print("Found %d observations" %len(flux_data))
        #l_ratio_ax.set_prop_cycle(cycler('color', colors))
        for index, color in enumerate(colors):
            if flux_data["xmm_obsid"][index]!="":
                    label=flux_data["epoch"][index]
                    if flux_data["nustar_obsid"][index]!="":
                        facemarkercolor="white"
                    else:
                        facemarkercolor=color
            else:
                    label=flux_data["epoch"][index]
                    facemarkercolor = "black"
            print("Plotting observation %s" %label)
            l_ratio_ax.errorbar(ratio_soft[index], constant * flux_data["%s" %total_band][index], xerr=[[ratio_err_low_soft[index]], 
                                                                                                            [ratio_err_high_soft[index]]], 
                                 yerr=[[constant * (flux_data["%s" %total_band]-flux_data["%s_lower"%total_band])[index]], 
                                       [constant * (flux_data["%s_upper" % total_band]-flux_data["%s" %total_band])[index]]], 
                                    label=label, markerfacecolor=facemarkercolor, color=color, marker=markers[index], markersize=10)
        l_ratio_ax.legend()
        outdir = "/home/agurpide/Documents/paper1/%s" % source_label
        if not os.path.isdir(outdir):
            os.mkdir(outdir)
        l_ratio_figure.savefig("%s/%s.png" % (outdir, plot_out_name), bbox_inches='tight') 

[('source_dir', '<U13'), ('model_dir', '<U30'), ('source_name', '<U18'), ('distance', '<f8'), ('hardness', '<f8')]
Plotting source holmbergX9
Creating color array for 10 datasets
Found 10 observations
Plotting observation 2002-04-10
Plotting observation 2002-04-16
Plotting observation 2004-09-26
Plotting observation 2011-03-24
Plotting observation 2011-09-26
Plotting observation 2011-11-23
Plotting observation 2012-10-23
Plotting observation 2012-10-25
Plotting observation 2012-11-12
Plotting observation 2012-11-14


## Frac vs time

In [13]:
plot_config = readbroadbandfile(file="broadband_fitting_plot.config")
if len(np.atleast_1d(plot_config))==1:
        plot_config = np.array([plot_config])
common_dir = "/home/agurpide/x_ray_data/"
plot_out_name = "frac_time"
param_2 = "FracSctr"
for source, source_label, model_dir, distance in zip(plot_config["source_dir"], plot_config["source_name"], plot_config["model_dir"], plot_config["distance"]):
    
        print("Plotting source %s" % source)
        frac_time_figure, frac_time_ax = create_plot("Date", 
                                           "Frac (%)",
                                           source_label)
        parsecs = distance * m.pow(10, 6)
        # parsecs to cm
        distancecm = u.pc.to(u.cm, parsecs)
        constant = 4 * m.pi * distancecm ** 2 / 10**39
        source_dir = "%s/%s/%s" %(common_dir, source, model_dir)
        if not os.path.isfile("%s/components/simpl_0.dat" % source_dir):
            print("Component not found")
            continue
        simple_data = read_comp_simpl("%s/components/simpl_0.dat" % source_dir)
        param_2err_low, param_2err_high, param_2lolimits, param_2uplimits = bounds_to_errors(simple_data["%s" %param_2], 
                                                                                         simple_data["%slow" %param_2], 
                                                                                         simple_data["%supp" %param_2])

        markers = get_markers_array(len(simple_data["epoch"]))
        print("Found %d observations" %len(simple_data))
        frac_time_ax.set_prop_cycle(cycler('color', colors))

        frac_time_ax.errorbar(Time(simple_data["epoch"]).jd, simple_data["%s" %param_2], marker=".", 
                              markersize=10, 
                                  yerr=[param_2err_low,
                       param_2err_high], lolims=param_2lolimits, uplims=param_2uplimits, ls='solid', color="black")

        date_formatter = FuncFormatter(jd_to_daymonthyear)
        plt.xticks(rotation=45)
        # set formatter
        frac_time_ax.xaxis.set_major_formatter(date_formatter)
        outdir = "/home/agurpide/Documents/paper1/%s" % source_label
        if not os.path.isdir(outdir):
            os.mkdir(outdir)
        frac_time_figure.savefig("%s/%s.png" % (outdir, plot_out_name), bbox_inches='tight') 

[('source_dir', '<U13'), ('model_dir', '<U30'), ('source_name', '<U18'), ('distance', '<f8'), ('hardness', '<f8')]
Plotting source NGC1313X-1
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Linked parameter found
Limit found
Limit found
Limit 